In [0]:
dbutils.widgets.text("api_key", "")

In [0]:
api_key = dbutils.widgets.get("api_key")

In [0]:
import requests
import datetime
import json

class Ingest:
    def __init__(self, api_key, base_currency="AUD", target_currencies=["USD", "JPY", "CNY"]):
        self.api_key = api_key
        self.base_currency = base_currency
        self.target_currencies = target_currencies
        self.base_data_dir = "/mnt/raw_data"

    def fetch_api_data(self, date_str):
        url = f"https://data.fixer.io/api/{date_str}"
        params = {
            "access_key": self.api_key,
            "base": self.base_currency,
            "symbols": ','.join(self.target_currencies)
        }
        response = requests.get(url, params=params)
        data = response.json()

        if not data.get("success", False):
            print(f"❌ Error fetching data for {date_str}: {data.get('error')}")
            return None
        return data

    def ingest_batch(self, start_date, end_date):
        current_date = datetime.datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")

        while current_date <= end_date:
            date_str = current_date.strftime("%Y-%m-%d")
            data = self.fetch_api_data(date_str)
            if data:
                file_path = f"/mnt/raw_data/{date_str}_{int(datetime.datetime.now().timestamp())}.json"
                dbutils.fs.put(f"dbfs:{file_path}", json.dumps(data), overwrite=True)
                print(f"✅ Saved data to {file_path}")

            current_date += datetime.timedelta(days=1)


dbutils.widgets.get("api_key")

ingest = Ingest(api_key)
ingest.ingest_batch(start_date="2025-02-20", end_date="2025-02-26")


Wrote 160 bytes.
✅ Saved data to /mnt/raw_data/2025-02-20_1742429783.json
Wrote 160 bytes.
✅ Saved data to /mnt/raw_data/2025-02-21_1742429783.json
Wrote 160 bytes.
✅ Saved data to /mnt/raw_data/2025-02-22_1742429784.json
Wrote 162 bytes.
✅ Saved data to /mnt/raw_data/2025-02-23_1742429784.json
Wrote 161 bytes.
✅ Saved data to /mnt/raw_data/2025-02-24_1742429784.json
Wrote 162 bytes.
✅ Saved data to /mnt/raw_data/2025-02-25_1742429785.json
Wrote 162 bytes.
✅ Saved data to /mnt/raw_data/2025-02-26_1742429785.json
